In [ ]:
%pip install rembg onnxruntime albumentations Pillow opencv-python "numpy<2"

In [ ]:
import os
import time
import cv2
import numpy as np
from PIL import Image
from rembg import remove
import albumentations as A
from albumentations.pytorch import ToTensorV2

#PIPELINE DE AUGMENTATION
transform = A.Compose([
    A.RandomResizedCrop(size=(224, 224), scale=(0.6, 1.0), p=1.0),

    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=25, p=0.6),

    A.OneOf([
        A.GaussNoise(var_limit=(0.01, 0.1)),
        A.GaussianBlur(blur_limit=(1, 3)),
    ], p=0.4),

    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.7),
    A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=12, val_shift_limit=10, p=0.4),

], p=1.0)

#FUNÇÕES AUXILIARES
def segment_leaf_rembg(image_path):
    """
    Remove o fundo da folha usando rembg.
    Retorna imagem PIL (com possível canal alfa).
    """
    input_img = Image.open(image_path).convert("RGB")
    output = remove(input_img)
    return output


def prepare_image_for_augmentation(pil_image):
    """
    Converte PIL -> numpy e garante formato RGB (3 canais).
    """
    img_np = np.array(pil_image)

    #Se tiver 4 canais (RGBA), converte para RGB
    if len(img_np.shape) == 3 and img_np.shape[-1] == 4:
        img_np = cv2.cvtColor(img_np, cv2.COLOR_RGBA2RGB)

    #Se for escala de cinza, converte para RGB
    elif len(img_np.shape) == 2 or (len(img_np.shape) == 3 and img_np.shape[-1] == 1):
        img_np = cv2.cvtColor(img_np, cv2.COLOR_GRAY2RGB)

    return img_np


def augment_save_alb_from_image(img, out_path, transform):
    """
    Aplica o pipeline de augmentation em uma imagem (numpy array)
    e salva a imagem resultante.
    """
    augmented = transform(image=img)
    out = augmented["image"]

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    cv2.imwrite(out_path, out)
    print("Imagem aumentada salva em:", out_path)


#FLUXO DE AUGMENTATION POR CLASSE
root_dir = "/Users/julia/Downloads/fundoruim_teste"  #Diretório raiz com imagens originais em subdiretórios
output_root = "/Users/julia/Downloads/photos_aug"    #Diretório onde as imagens transformadas serão salvas
print("Iniciando augmentation...")

for label in os.listdir(root_dir):
    class_path = os.path.join(root_dir, label)
    if not os.path.isdir(class_path):
        print(f"Pulado (não é diretório): {class_path}")
        continue

    out_class_dir = os.path.join(output_root, label + "_aug")
    os.makedirs(out_class_dir, exist_ok=True)

    print(f"Processando classe: {label}")
    for img_name in os.listdir(class_path):
        if img_name.lower().endswith(("jpg", "bmp", "png")):
            img_path = os.path.join(class_path, img_name)
            out_path = os.path.join(out_class_dir, img_name)

            try:
                print(f"Aplicando augmentation em: {img_name}")
                start_rembg = time.time()
                result_rembg = segment_leaf_rembg(img_path)
                end_rembg = time.time()
                print(f"    Fundo removido em {end_rembg - start_rembg:.2f}s")

                #Converte e garante formato RGB
                result_rembg_np = prepare_image_for_augmentation(result_rembg)

                #Aplica augmentation e salva
                augment_save_alb_from_image(result_rembg_np, out_path, transform)

            except Exception as e:
                print(f"[!] Erro ao processar {img_path}: {e}")

print("[!] Augmentation concluída!")

In [ ]:
pip install --force-reinstall "numpy<2"

In [ ]:
import numpy, cv2
print("NumPy OK:", numpy.__version__)
print("OpenCV OK:", cv2.__version__)